In [1]:
import findspark
findspark.init()
import pyspark

conf = pyspark.SparkConf()
conf.set("spark.driver.bindAddress", "127.0.0.1")  # Localhost
conf = conf.setMaster("local[*]")
conf = conf.setAppName("App")

sc = pyspark.SparkContext(conf=conf)




Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 10:42:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#Einlesen der Flugdaten von einem Tag + entfernen des ersten Datensatzes (Header)
myFileRDD = sc.textFile('/Users/I750252/Downloads/20230126_merged.csv')
myFileRDD.take(8)
header = myFileRDD.first()

dataRDD = myFileRDD.filter(lambda line: line != header)

#Anzahl an Datensätzen
amount_of_data = dataRDD.count()
print(amount_of_data)
dataRDD.take(8)

1389565


['"1","2023/01/26 01:59:25+00","3c3f79",51.2953491210938,6.78500407450908,NA,NA,NA,"E03",1,0,0,NA,NA,NA,1674698360.285,1674698360.285,"[ -1408232379 ]","2023/01/26 01:00:00+00",59',
 '"2","2023/01/26 01:02:36+00","4b5c48",47.44482421875,8.56004905700684,NA,NA,NA,"EUGEN15",1,0,0,NA,NA,NA,1674694955.854,1674694955.854,"[ 1996109518, 1996107667, 1208480659 ]","2023/01/26 01:00:00+00",119',
 '"3","2023/01/26 01:03:36+00","4b5c48",47.4540367126465,8.55378341674805,NA,NA,NA,"EUGEN15",1,0,0,NA,NA,NA,1674695011.733,1674695011.733,"[ -1408234159, 1996109518, 1208480659 ]","2023/01/26 01:00:00+00",179',
 '"4","2023/01/26 01:04:36+00","4b5c48",47.4563986568128,8.55819115271935,NA,NA,NA,"EUGEN15",1,0,0,NA,NA,NA,1674695041.209,1674695041.209,"[ ]","2023/01/26 01:00:00+00",239',
 '"5","2023/01/26 01:05:36+00","4b5c48",47.4563986568128,8.55819115271935,NA,NA,NA,"EUGEN15",1,0,0,NA,NA,NA,1674695041.209,1674695041.209,"[ ]","2023/01/26 01:00:00+00",299',
 '"6","2023/01/26 01:06:36+00","4b5c48",47.456398

In [3]:
# Funktion zum Parsen einer Zeile
def parse_line(line):
    parts = line.split(',')
    return (
        parts[2].strip('"'),  # Key: Flugzeug-ID (icao24)
        {  # Value: Alle relevanten Daten als Dictionary
            "time": parts[1].strip('"'),
            "lat": float(parts[3].strip('"')),
            "lon": float(parts[4].strip('"')),
            "onground": int(parts[9].strip('"'))
        }
    )

# RDD transformieren: Gruppiere die Daten nach Flugzeug-ID
flights_rdd = dataRDD.map(parse_line).groupByKey().mapValues(list)

#Anzahl der Flugzeuge, die innerhalb einer Stunde im französischen Luftraum aktiv waren
amount_of_aircrafts = flights_rdd.count()
print(amount_of_aircrafts)

print(flights_rdd.take(2))


6938
[('4b5c48', [{'time': '2023/01/26 01:02:36+00', 'lat': 47.44482421875, 'lon': 8.56004905700684, 'onground': 1}, {'time': '2023/01/26 01:03:36+00', 'lat': 47.4540367126465, 'lon': 8.55378341674805, 'onground': 1}, {'time': '2023/01/26 01:04:36+00', 'lat': 47.4563986568128, 'lon': 8.55819115271935, 'onground': 1}, {'time': '2023/01/26 01:05:36+00', 'lat': 47.4563986568128, 'lon': 8.55819115271935, 'onground': 1}, {'time': '2023/01/26 01:06:36+00', 'lat': 47.4563986568128, 'lon': 8.55819115271935, 'onground': 1}, {'time': '2023/01/26 01:07:36+00', 'lat': 47.4563986568128, 'lon': 8.55819115271935, 'onground': 1}, {'time': '2023/01/26 01:08:36+00', 'lat': 47.4563986568128, 'lon': 8.55819115271935, 'onground': 1}, {'time': '2023/01/26 01:09:36+00', 'lat': 47.4576788433528, 'lon': 8.5554445706881, 'onground': 1}, {'time': '2023/01/26 01:10:36+00', 'lat': 47.4585284216929, 'lon': 8.54356032151442, 'onground': 1}, {'time': '2023/01/26 01:11:36+00', 'lat': 47.4589576721191, 'lon': 8.5374412

In [4]:
#Einlesen der Flugzeug-Typ Daten + entfernen des ersten Datensatzes (Header)
myTypeRDD = sc.textFile('/Users/I750252/Downloads/aircraft-type.csv')
myTypeRDD.take(8)
header = myTypeRDD.first()

typeRDD = myTypeRDD.filter(lambda line: line != header)
typeRDD.take(10)

["'000000','2017-10-19 18:30:18',0,0,,'',,'',,,'','','','',unknow,0,'','','','','','','',,,'','','','','',0",
 "'000001','2016-11-27 00:10:39',0,0,,'',,'',,,L1P,'',SHIJIAZHUANG,'',Antonov 2,0,'','','','','',Private,'',,,SP-FGR,'','','',AN2,0",
 "'000002','2018-05-29 02:00:00',0,0,,Light (< 15500 lbs),,'',,,'','','','','',0,'','','','','','','',,,'','','','','',0",
 "'000003','2018-04-01 02:00:00',0,0,,No ADS-B Emitter Category Information,,'',,,'','','','','',0,'','','','','','','',,,'','','','','',0",
 "'000004','2018-05-29 02:00:00',0,0,,No ADS-B Emitter Category Information,,'',,,'','','','','',0,'','','','','','','',,,'','','','','',0",
 "'000005','1970-01-01 01:00:00',0,0,,'',,'',,,'','','','','',0,'','','','','','','',,,'','','','','',0",
 "'000006','2017-09-01 02:00:00',0,0,,No ADS-B Emitter Category Information,,'',,,'','','','','',0,'','','','','','','',,,'','','','','',0",
 "'000007','2018-01-01 01:00:00',0,0,,No ADS-B Emitter Category Information,,'',,,'','','','','',0,'',''

In [10]:
def parse_type(line):
    parts = line.split(',')
    # Baseline-Verarbeitung ohne unnötige Komplexität
    return (
        parts[0].strip("'"),  # Flugzeug-ID (icao24)
        {
            "description": parts[5].strip("'") if len(parts) > 4 else "",
            "manufacturer": parts[13].strip("'") if len(parts) > 12 else "",
            "model": parts[14].strip("'") if len(parts) > 14 else "",
            "type": parts[15].strip("'") if len(parts) > 15 else "",
            "airline": parts[18].strip("'") if len(parts) > 16 else ""
        }
    )


# RDD transformieren: Ausgewählte Daten beziehen
cleaned_type_rdd = typeRDD.map(parse_type)
cleaned_type_rdd.take(5)
print(cleaned_type_rdd.count())

info = cleaned_type_rdd.filter(lambda x: x[0] == "3c48a3").collect()
print(info)



601337
[('3c48a3', {'description': '', 'manufacturer': 'Boeing', 'model': 'Boeing 737 330', 'type': '0', 'airline': 'LUFTHANSA'})]


In [16]:
# Beschreibung für jeden Datensatz hinzufügen
result_rdd = flights_rdd.leftOuterJoin(cleaned_type_rdd)
result_rdd.take(1)
# Transformiere das Ergebnis, um die Daten zu kombinieren
# Behalte die Beschreibung zuerst, gefolgt von den Flugdaten
combined_rdd = result_rdd.map(lambda x: (
    x[0],  # Flugzeug-ID
    {
        "aircraft_info": x[1][1] if x[1][1] else {},  # Beschreibung, falls vorhanden
        "flight_data": x[1][0]  # Flugdaten
    }
))

# Ergebnisse anzeigen
combined_rdd.take(10)

[('3964e4',
  {'aircraft_info': {'description': '',
    'manufacturer': 'Boeing',
    'model': '737NG 8K2/W',
    'type': '0',
    'airline': 'FRENCH SUN'},
   'flight_data': [{'time': '2023/01/26 05:20:32+00',
     'lat': 48.7257499694824,
     'lon': 2.36079289362981,
     'onground': 1},
    {'time': '2023/01/26 05:21:32+00',
     'lat': 48.725517402261,
     'lon': 2.36079567357113,
     'onground': 1},
    {'time': '2023/01/26 05:22:32+00',
     'lat': 48.7249488830566,
     'lon': 2.36077528733474,
     'onground': 1},
    {'time': '2023/01/26 05:23:32+00',
     'lat': 48.7248916625977,
     'lon': 2.36132108248197,
     'onground': 1},
    {'time': '2023/01/26 05:24:32+00',
     'lat': 48.7248916625977,
     'lon': 2.36132108248197,
     'onground': 1},
    {'time': '2023/01/26 05:25:32+00',
     'lat': 48.7248889470504,
     'lon': 2.36131969251131,
     'onground': 1},
    {'time': '2023/01/26 05:26:32+00',
     'lat': 48.7248001098633,
     'lon': 2.36082810621995,
     'ongr

In [18]:
#Ermittlung der Anzahl an Flug-Datensätze pro Flugzeug
countdatasetsRDD = combined_rdd.map(lambda x: (x[0],len(x[1]['flight_data']))).sortBy(lambda x: x[1], ascending=False)
print(countdatasetsRDD.take(10))


# Bestimmung Flugstatus
airRDD = combined_rdd.map(lambda x: (x[0], x[1]['flight_data'][-1]["onground"]))
onground = airRDD.filter(lambda x: x[1]==1).count()
inair = airRDD.filter(lambda x: x[1]==0).count()

print("Flugobjekte aktuell in der Luft: " + str(onground))
print("Flugobjekte aktuell am Boden: " + str(inair))


# Ermittlung wie viele nur in der Luft waren, wie viele nur am Boden und wie viele beides waren an einem Tag
nextRDD = combined_rdd.map(lambda x: (x[0], sum(datapoint["onground"] for datapoint in x[1]['flight_data']), len(x[1]['flight_data'])))
print(nextRDD.take(1))
#Quotienten berechnen um zu schauen wer gelandet ist und wer nicht
calculatedRDD = nextRDD.map(lambda x: (x[0], x[1]/x[2]))
print(calculatedRDD.take(1))

not_started = calculatedRDD.filter(lambda x: x[1] == 1).count()
print(f"Es sind so viele Objekte nicht am Boden geblieben: {not_started}")
not_landed = calculatedRDD.filter(lambda x: x[1] == 0).count()
print(f"Es sind so viele Objekte nicht gelandet: {not_landed}")
startet_or_landed = calculatedRDD.filter(lambda x: x[1] < 1 and x[1] >0).count()
print(f"Es sind so viele Objekte gestartet oder gelandet: {startet_or_landed}")


# Anzahl Starts und Anzahl Landungen

[('392adb', 1440), ('490031', 1439), ('425851', 1439), ('425850', 1437), ('344043', 1437), ('425855', 1437), ('490020', 1437), ('392afb', 1435), ('425852', 1435), ('425854', 1427)]
Flugobjekte aktuell in der Luft: 1779
Flugobjekte aktuell am Boden: 5159
[('3964e4', 43, 482)]
[('3964e4', 0.08921161825726141)]
Es sind so viele Objekte nicht am Boden geblieben: 603
Es sind so viele Objekte nicht gelandet: 2201
Es sind so viele Objekte gestartet oder gelandet: 4134


In [19]:
import folium
from IPython.display import display

# Die Flugzeug-ID, die visualisiert werden soll
selected_icao24 = "0200ec"

# Filtere die Daten für die spezifische Flugzeug-ID im RDD
filtered_data = combined_rdd.filter(lambda x: x[0] == selected_icao24).collect()

if filtered_data:
    # Hole die Daten für das ausgewählte Flugzeug
    icao24, route_data = filtered_data[0]
    aircraft_info = route_data["aircraft_info"]  # Zugriff auf die Flugzeugbeschreibung
    flight_data = route_data["flight_data"]  # Zugriff auf die Flugdaten

    print(f"Flugzeug-ID: {icao24}")
    print(f"Flugzeugtyp: {aircraft_info['type']}")  # Flugzeugtyp anzeigen

    # Erstelle die Karte, zentriert auf den ersten Datenpunkt
    first_lat = flight_data[0]["lat"]
    first_lon = flight_data[0]["lon"]
    mymap = folium.Map(location=[first_lat, first_lon], zoom_start=12)

    # Zeichne Linien zwischen aufeinanderfolgenden Punkten
    for i in range(len(flight_data) - 1):
        point_a = flight_data[i]
        point_b = flight_data[i + 1]

        # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
        if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
            folium.PolyLine(
                locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                color="blue",
                weight=2.5,
                opacity=0.8
            ).add_to(mymap)

    # Füge Marker nur für Landepunkte hinzu
    for point in flight_data:
        if point["onground"] == 1:  # Nur Landepunkte anzeigen
            folium.Marker(
                location=(point["lat"], point["lon"]),
                popup=f"Zeit: {point['time']}<br>Am Boden: {point['onground']}<br>Hersteller: {aircraft_info['manufacturer']}<br>Flugzeugtyp: {aircraft_info['model']}",
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(mymap)

    # Zeige die Karte im Notebook
    display(mymap)
else:
    print(f"Keine Daten für Flugzeug-ID '{selected_icao24}' gefunden.")


Flugzeug-ID: 0200ec
Flugzeugtyp: 0


In [20]:
import folium
from IPython.display import display
import random

# Zufällige Farben für die Flugzeuge
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Nimm eine beliebige Anzahl zufälliger Flugzeuge aus dem RDD
random_aircrafts = combined_rdd.takeSample(withReplacement=False, num=10)

# Erstelle eine Karte, zentriert auf einen Beispielpunkt
if random_aircrafts:
    # Hole den ersten Punkt zur Zentrierung der Karte
    first_lat = random_aircrafts[0][1]["flight_data"][0]["lat"]
    first_lon = random_aircrafts[0][1]["flight_data"][0]["lon"]
    mymap = folium.Map(location=[first_lat, first_lon], zoom_start=6)

    # Iteriere über die zufälligen Flugzeuge
    for icao24, data in random_aircrafts:
        aircraft_info = data["aircraft_info"]  # Infos über das Flugzeug
        flight_data = data["flight_data"]      # Flugdaten

        # Sichere Abfrage von Typ, Modell und Hersteller mit Standardwerten
        aircraft_type = aircraft_info.get("type", "Unbekannt")
        aircraft_model = aircraft_info.get("model", "Unbekannt")
        aircraft_manufacturer = aircraft_info.get("manufacturer", "Unbekannt")

        # Generiere eine zufällige Farbe für die Linien
        color = get_random_color()

        # Zeichne Linien zwischen aufeinanderfolgenden Punkten
        for i in range(len(flight_data) - 1):
            point_a = flight_data[i]
            point_b = flight_data[i + 1]

            # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
            if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
                folium.PolyLine(
                    locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                    color=color,
                    weight=2.5,
                    opacity=0.8
                ).add_to(mymap)

        # Füge Marker nur für Landepunkte hinzu
        for point in flight_data:
            if point["onground"] == 1:  # Nur Landepunkte anzeigen
                folium.Marker(
                    location=(point["lat"], point["lon"]),
                    popup=(
                        f"Flugzeug: {icao24}<br>"
                        f"Hersteller: {aircraft_manufacturer}<br>"
                        f"Modell: {aircraft_model}<br>"
                        f"Zeit: {point['time']}<br>"
                        f"Am Boden: {point['onground']}"
                    ),
                    icon=folium.Icon(color="green", icon="info-sign")
                ).add_to(mymap)

    # Zeige die Karte im Notebook
    display(mymap)
else:
    print("Keine Daten gefunden.")


In [ ]:
# Darstellung aller Flugdaten als Heatmap
import folium
from folium.plugins import HeatMap

# Reduziere die Daten durch Sampling (alle 10 Minuten)
# Ein Datensatz
# reduced_data = grouped_rdd.take(1)

reduced_data = combined_rdd.map(lambda x: (
    x[0],  # Flugzeug-ID
    x[1]["flight_data"][::10]  # Wähle jeden 10. Datensatz aus flight_data
))

# Sammle und bereite die Heatmap-Daten vor
heatmap_data = [
    (point["lat"], point["lon"])
    for _, route_data in reduced_data.collect()
    for point in route_data
    if point["lat"] is not None and point["lon"] is not None
]

# Erstelle die Karte
if heatmap_data:
    mymap = folium.Map(location=[heatmap_data[0][0], heatmap_data[0][1]], zoom_start=6)
    HeatMap(heatmap_data, radius=10).add_to(mymap)
    from IPython.display import display
    display(mymap)
else:
    print("Keine gültigen Punkte für die Heatmap gefunden.")


24/11/28 10:49:10 ERROR Executor: Exception in task 0.0 in stage 159.0 (TID 960)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/var/folders/l3/x8z60lyd2yj23s6rjdy757dr0000gn/T/ipykernel_53228/1825436184.py", line 11, in <lambda>
KeyError: slice(None, None, 10)

	at org.apache.spark.api.python.BasePythonRun

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 159.0 failed 1 times, most recent failure: Lost task 0.0 in stage 159.0 (TID 960) (dy2g339fmm.fritz.box executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args

24/11/28 10:49:10 WARN TaskSetManager: Lost task 10.0 in stage 159.0 (TID 970) (dy2g339fmm.fritz.box executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 0 in stage 159.0 failed 1 times, most recent failure: Lost task 0.0 in stage 159.0 (TID 960) (dy2g339fmm.fritz.box executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    retur